In [1]:
!pip install iterative-stratification

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/운동동작분류AI경진대회')

In [4]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

import os, gc, random, datetime
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from joblib import dump, load
from time import time
import scipy as sp
import scipy.fftpack

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.4.1


# Preprocessing

In [5]:
# 데이터 불러오기

path = './data/'
train = pd.read_csv(path + 'train_features.csv')
train_label = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')

In [6]:
# Pre-Processing Effect on the Accuracy of Event-Based Activity Segmentation and Classification through Inertial Sensors 
# https://www.researchgate.net/publication/281836367_Pre-Processing_Effect_on_the_Accuracy_of_Event-Based_Activity_Segmentation_and_Classification_through_Inertial_Sensors

train['acc_t'] = train.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 + x['acc_z'] **2)**(1/2), axis=1)
test['acc_t'] = test.apply(lambda x : (x['acc_x']**2 + x['acc_y'] **2 + x['acc_z'] **2)**(1/2), axis=1)
train['gy_t'] = train.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 + x['gy_z'] **2)**(1/2), axis=1)
test['gy_t'] = test.apply(lambda x : (x['gy_x']**2 + x['gy_y'] **2 + x['gy_z'] **2)**(1/2), axis=1)

# SVM selected features
train['mean'] = train[['acc_x','acc_y']].mean(axis=1)
train['median'] = train[['acc_y', 'gy_z', 'gy_t']].median(axis=1)
train['standard_deviation'] = train[['acc_x', 'acc_y']].std(axis=1)
train['interquartile'] = train.quantile(.75, axis=1) - train.quantile(.25, axis=1)

test['mean'] = test[['acc_x','acc_y']].mean(axis=1)
test['median'] = test[['acc_y', 'gy_z', 'gy_t']].median(axis=1)
test['standard_deviation'] = test[['acc_x', 'acc_y']].std(axis=1)
test['interquartile'] = test.quantile(.75, axis=1) - test.quantile(.25, axis=1)

In [ ]:
x = np.array(train.iloc[:,2:]).reshape(-1, 600, 12)
y = tf.keras.utils.to_categorical(train_label['label'])
test = np.array(test.iloc[:,2:]).reshape(-1, 600, 12)

## 도비님의 데이터 증강 방법

In [ ]:
# 26번을 제외한 id 리스트
feature = list(train_label[train_label['label'] != 26]['id'])

In [ ]:
# train 데이터에서 26번을 삭제시킨다.
temp = []
for n in tqdm(range(train.shape[0])):
    if train['id'][n] in feature:
        temp.append(train.iloc[n])

In [ ]:
# 26번을 삭제시킨 데이터프레임
without = pd.DataFrame(data=np.array(temp), columns=train.columns)
without = without.astype({'id':int, 'time':int})

In [ ]:
without_train = np.array(without.iloc[:,2:]).reshape(-1, 600, 12)
without_label = train_label[train_label['label'] != 26]['label']

In [ ]:
def aug(data, shift):
    shift_data = np.roll(data, shift, axis=1)
    return shift_data

# 데이터 증강
shift_data = []
shift_label = []
for n in tqdm(range(1, 10)):
    shifted = aug(without_train, n*60)
    shift_data.append(shifted)
    shift_label.append(without_label)

shift_data = np.array(shift_data).reshape(-1,600,12)
shift_label = np.array(shift_label).reshape(-1,1)
shift_categorical = tf.keras.utils.to_categorical(shift_label)

In [ ]:
# 원본 데이터와 증강 데이터 합치기
concat_train = np.concatenate((x, shift_data), axis=0)
concat_label = np.concatenate((y, shift_categorical), axis=0)
print(concat_train.shape)
print(concat_label.shape)

(17588, 600, 12)
(17588, 61)


In [ ]:
del x, shift_data, y, shift_categorical, without, without_train, without_label, feature, temp

# Training

Base Transformer structure from https://www.tensorflow.org/tutorials/text/transformer, modified with Swish activation function.

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b = True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        
        scaled_attention_logits += (mask * -1e9)  

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis = -1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads):
        
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm = [0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation = 'relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

class EncoderLayer(tf.keras.layers.Layer):
    
    def __init__(self, d_model, num_heads, dff, rate = 0.1):
        
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training = training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):
    
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 maximum_position_encoding, rate = 0.1):
        
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.dff = dff
        self.maximum_position_encoding = maximum_position_encoding
        self.rate = rate

#         self.pos_encoding = positional_encoding(self.maximum_position_encoding, 
#                                                 self.d_model)
#         self.embedding = tf.keras.layers.Dense(self.d_model)
        self.pos_emb = tf.keras.layers.Embedding(input_dim = self.maximum_position_encoding, 
                                                 output_dim = self.d_model)

        self.enc_layers = [EncoderLayer(self.d_model, self.num_heads, self.dff, self.rate) 
                           for _ in range(self.num_layers)]

        self.dropout = tf.keras.layers.Dropout(self.rate)
        
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'maximum_position_encoding': self.maximum_position_encoding,
            'dropout': self.dropout,
        })
        return config

    def call(self, x, training, mask = None):

        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding.
#         x += self.pos_encoding[:, :seq_len, :]
#         x = self.embedding(x)
        positions = tf.range(start = 0, limit = seq_len, delta = 1)
        x += self.pos_emb(positions)

        x = self.dropout(x, training = training)

        for i in range(self.num_layers):

            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [ ]:
def create_transformer_model(num_columns, num_labels, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate):
    
    inp = tf.keras.layers.Input(shape = (window_size, num_columns))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dense(d_model)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.SpatialDropout1D(dropout_rate)(x)
    x = TransformerEncoder(num_layers, d_model, num_heads, dff, window_size, dropout_rate)(x)
    out = tf.keras.layers.Dense(num_labels, activation = 'softmax')(x[:, -1, :])
    
    model = tf.keras.models.Model(inputs = inp, outputs = out)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['AUC'])
    
    return model

In [ ]:
batch_size = 32
num_layers = 1
d_model = 128
num_heads = 1
dff = 128
window_size = 600
dropout_rate = 0.15
weight_decay = 0
label_smoothing = 1e-2
learning_rate = 1e-3
verbose = 1

In [ ]:
# 모델 1번: Transformer

def build_transformer(split_num, train, target, test, rnd):
    start_time_fold = time()
    # return train pred prob and test pred prob 
    test_pred = np.zeros((test.shape[0], 61))

    ckp_path = 'transformer.hdf5'

    rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 3, verbose = verbose, min_delta = 1e-4, mode = 'min')
    ckp = ModelCheckpoint(ckp_path, monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')
    es = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 4, mode = 'min', baseline = None, restore_best_weights = True, verbose = 0)

    mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in mskf.split(train, target):

        # split train, validation set
        X = train[train_idx]
        y = target[train_idx]
        valid_x = train[val_idx]
        valid_y = target[val_idx]

        #가벼운 모델 생성
        model = create_transformer_model(train.shape[2], 61, num_layers, d_model, num_heads, dff, window_size, dropout_rate, weight_decay, label_smoothing, learning_rate)

        model.fit(X, y, epochs = 100,
                  validation_data = (valid_x, valid_y),
                  batch_size = batch_size,
                  callbacks = [rlr, ckp, es],
                  verbose = verbose)
        
        # save feat
        model.load_weights(ckp_path)

        # 테스트 데이터 증강
        shift_test = []
        for n in range(10):
            shifted = aug(test, n*60)
            test_pred += model.predict(shifted)/(split_num*10)
        
        # release
        del model
        gc.collect()
        print('  ==============================================================================================  ')

        
    return test_pred

transformer_train1, transformer_test1 = build_transformer(5, concat_train, concat_label, test, 1)
transformer_train2, transformer_test2 = build_transformer(5, concat_train, concat_label, test, 2)

Epoch 1/100
440/440 [==============================] - 15s 30ms/step - loss: 3.1127 - auc: 0.8338 - val_loss: 1.7794 - val_auc: 0.9600
Epoch 2/100
440/440 [==============================] - 13s 29ms/step - loss: 1.9249 - auc: 0.9562 - val_loss: 1.3764 - val_auc: 0.9763
Epoch 3/100
440/440 [==============================] - 13s 30ms/step - loss: 1.6059 - auc: 0.9694 - val_loss: 1.2295 - val_auc: 0.9801
Epoch 4/100
440/440 [==============================] - 13s 29ms/step - loss: 1.4120 - auc: 0.9758 - val_loss: 1.0556 - val_auc: 0.9849
Epoch 5/100
440/440 [==============================] - 13s 29ms/step - loss: 1.2631 - auc: 0.9808 - val_loss: 0.9872 - val_auc: 0.9873
Epoch 6/100
440/440 [==============================] - 13s 29ms/step - loss: 1.2017 - auc: 0.9825 - val_loss: 0.8578 - val_auc: 0.9915
Epoch 7/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1077 - auc: 0.9845 - val_loss: 0.8202 - val_auc: 0.9914
Epoch 8/100
440/440 [==============================] - 


Epoch 1/100
440/440 [==============================] - 15s 31ms/step - loss: 3.1020 - auc: 0.8366 - val_loss: 1.8115 - val_auc: 0.9630
Epoch 2/100
440/440 [==============================] - 13s 30ms/step - loss: 2.0098 - auc: 0.9511 - val_loss: 1.3679 - val_auc: 0.9775
Epoch 3/100
440/440 [==============================] - 13s 29ms/step - loss: 1.6392 - auc: 0.9679 - val_loss: 1.1351 - val_auc: 0.9836
Epoch 4/100
440/440 [==============================] - 13s 29ms/step - loss: 1.4521 - auc: 0.9723 - val_loss: 0.9991 - val_auc: 0.9881
Epoch 5/100
440/440 [==============================] - 13s 30ms/step - loss: 1.3102 - auc: 0.9784 - val_loss: 0.9621 - val_auc: 0.9881
Epoch 6/100
440/440 [==============================] - 13s 30ms/step - loss: 1.1943 - auc: 0.9825 - val_loss: 0.8560 - val_auc: 0.9906
Epoch 7/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1052 - auc: 0.9847 - val_loss: 0.8334 - val_auc: 0.9911
Epoch 8/100
440/440 [==============================] -


Epoch 1/100
440/440 [==============================] - 15s 31ms/step - loss: 3.0901 - auc: 0.8367 - val_loss: 1.8356 - val_auc: 0.9588
Epoch 2/100
440/440 [==============================] - 13s 29ms/step - loss: 1.9623 - auc: 0.9533 - val_loss: 1.3354 - val_auc: 0.9780
Epoch 3/100
440/440 [==============================] - 13s 29ms/step - loss: 1.6016 - auc: 0.9672 - val_loss: 1.1802 - val_auc: 0.9827
Epoch 4/100
440/440 [==============================] - 13s 30ms/step - loss: 1.4118 - auc: 0.9750 - val_loss: 1.0491 - val_auc: 0.9871
Epoch 5/100
440/440 [==============================] - 13s 29ms/step - loss: 1.3130 - auc: 0.9796 - val_loss: 1.0026 - val_auc: 0.9868
Epoch 6/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1739 - auc: 0.9831 - val_loss: 0.8699 - val_auc: 0.9895
Epoch 7/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1014 - auc: 0.9850 - val_loss: 0.8397 - val_auc: 0.9911
Epoch 8/100
440/440 [==============================] -


Epoch 1/100
440/440 [==============================] - 15s 30ms/step - loss: 3.0787 - auc: 0.8417 - val_loss: 1.8675 - val_auc: 0.9584
Epoch 2/100
440/440 [==============================] - 13s 30ms/step - loss: 1.9414 - auc: 0.9559 - val_loss: 1.3733 - val_auc: 0.9761
Epoch 3/100
440/440 [==============================] - 13s 29ms/step - loss: 1.6113 - auc: 0.9683 - val_loss: 1.1937 - val_auc: 0.9812
Epoch 4/100
440/440 [==============================] - 13s 29ms/step - loss: 1.3836 - auc: 0.9755 - val_loss: 1.0156 - val_auc: 0.9866
Epoch 5/100
440/440 [==============================] - 13s 29ms/step - loss: 1.2843 - auc: 0.9789 - val_loss: 0.9422 - val_auc: 0.9884
Epoch 6/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1670 - auc: 0.9829 - val_loss: 0.8551 - val_auc: 0.9906
Epoch 7/100
440/440 [==============================] - 13s 29ms/step - loss: 1.1119 - auc: 0.9838 - val_loss: 0.7707 - val_auc: 0.9910
Epoch 8/100
440/440 [==============================] -


Epoch 1/100
440/440 [==============================] - 15s 31ms/step - loss: 3.0140 - auc: 0.8486 - val_loss: 1.7700 - val_auc: 0.9623
Epoch 2/100
440/440 [==============================] - 13s 30ms/step - loss: 1.9675 - auc: 0.9528 - val_loss: 1.3447 - val_auc: 0.9810
Epoch 3/100
440/440 [==============================] - 13s 30ms/step - loss: 1.5591 - auc: 0.9697 - val_loss: 1.1718 - val_auc: 0.9832
Epoch 4/100
440/440 [==============================] - 13s 29ms/step - loss: 1.4104 - auc: 0.9771 - val_loss: 1.0134 - val_auc: 0.9890
Epoch 5/100
440/440 [==============================] - 13s 29ms/step - loss: 1.2923 - auc: 0.9796 - val_loss: 0.9371 - val_auc: 0.9891
Epoch 6/100
440/440 [==============================] - 13s 30ms/step - loss: 1.1939 - auc: 0.9832 - val_loss: 0.8852 - val_auc: 0.9908
Epoch 7/100
440/440 [==============================] - 13s 30ms/step - loss: 1.1063 - auc: 0.9856 - val_loss: 0.8048 - val_auc: 0.9928
Epoch 8/100
440/440 [==============================] -

In [ ]:
sample_submssion = pd.read_csv(path + 'sample_submission.csv')
sample_submssion.iloc[:,1:] = transformer_test_shifted
sample_submssion.to_csv("transformer_shifted.csv", index = False)
sample_submssion

In [ ]:
# https://www.kaggle.com/gogo827jz/jane-street-ffill-transformer-baseline
# https://wikidocs.net/31379
# https://www.tensorflow.org/tutorials/text/transformer